In [68]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

In [53]:
def convert_to_binary(x_train):
    x_train_new = []
    for j in range(len(x_train)):
        array = x_train[j]
        bin = np.vectorize(np.binary_repr)(array, width=8)
        new_bin = np.zeros((len(bin), 8))
        for i in range(len(bin)):
            new_bin[i] = np.array([int(x) for x in bin[i]])
        x_train_new.append(new_bin)
    return np.array(x_train_new)


In [60]:
# Load mnist data
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# flatten the images
x_train = x_train.reshape(-1, 784).astype(int)
x_test = x_test.reshape(-1, 784).astype(int)

x_train_binary = convert_to_binary(x_train)
x_test_binary = convert_to_binary(x_test)

(60000, 28, 28)
(60000, 784, 8)


In [106]:
# store the training and testing data in a csv file
np.save("mnist_data/mnist_train", x_train_binary)
np.save("mnist_data/mnist_test", x_test_binary)
np.save("mnist_data/mnist_train_labels", y_train)
np.save("mnist_data/mnist_test_labels", y_test)

In [69]:
# Define model architecture
model = models.Sequential([
    layers.Reshape((28, 28, 8), input_shape=(784, 8)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')  # Assuming you have 10 classes in your dataset
])

# Compile model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train model
model.fit(x_train_binary, y_train, epochs=10, batch_size=32, validation_split=0.2)


Epoch 1/10
1500/1500 [==============================] - 36s 23ms/step - loss: 0.1526 - accuracy: 0.9521 - val_loss: 0.0678 - val_accuracy: 0.9802
Epoch 2/10
1500/1500 [==============================] - 35s 23ms/step - loss: 0.0502 - accuracy: 0.9841 - val_loss: 0.0587 - val_accuracy: 0.9823
Epoch 3/10
1500/1500 [==============================] - 35s 23ms/step - loss: 0.0341 - accuracy: 0.9889 - val_loss: 0.0539 - val_accuracy: 0.9852
Epoch 4/10
1500/1500 [==============================] - 35s 24ms/step - loss: 0.0250 - accuracy: 0.9919 - val_loss: 0.0531 - val_accuracy: 0.9855
Epoch 5/10
1500/1500 [==============================] - 41s 27ms/step - loss: 0.0184 - accuracy: 0.9938 - val_loss: 0.0464 - val_accuracy: 0.9890
Epoch 6/10
1500/1500 [==============================] - 37s 25ms/step - loss: 0.0145 - accuracy: 0.9955 - val_loss: 0.0527 - val_accuracy: 0.9879
Epoch 7/10
1500/1500 [==============================] - 37s 25ms/step - loss: 0.0117 - accuracy: 0.9962 - val_loss: 0.0662 -

In [81]:
# Assuming your model is already trained and named 'model'
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(x_test_binary, y_test)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)


313/313 [==============================] - 2s 7ms/step - loss: 0.0533 - accuracy: 0.9861
Test Loss: 0.05330564081668854
Test Accuracy: 0.9861000180244446


In [101]:
# save the model
model.save('mnist_binary.h5')

In [102]:
# load the model
model = models.load_model('mnist_binary.h5')